In [1]:
import glob
import json
import os
import requests
import json
from collections import Counter as counter
import copy

In [2]:
creds = json.load(
        open('env_vars.json'))

token = creds['api_token']

In [3]:
create_group_url = 'https://ballchasing.com/api/groups'
body = {
    "name": "23_aug",
    "parent": "egrls-jh5f8i5yv7",
    "player_identification": "by-id",
    "team_identification": "by-player-clusters"
}
req = requests.post(create_group_url, headers={
                      'Authorization': token, 'Content-Type': "application/json"}, json=body)

group_link = json.loads(req.text)['id']
group_link

'23_aug-5o00103ovf'

In [3]:
upload_url = "https://ballchasing.com/api/v2/upload?visibility=public&group=21_aug-2upgrsk4ns"# + group_link
get_group_url = "https://ballchasing.com/api/replays/?group=21_aug-2upgrsk4ns"# + group_link

get_group_url, upload_url

('https://ballchasing.com/api/replays/?group=21_aug-2upgrsk4ns',
 'https://ballchasing.com/api/v2/upload?visibility=public&group=21_aug-2upgrsk4ns')

In [4]:
def upload(path, token):
    files = {'file': open(path, 'rb')}
    r = requests.post(upload_url, headers={
                      'Authorization': token}, files=files)

    if r.status_code == 201:
        # replay successfully created, return it's id
        return r.json()['id']
    elif r.status_code == 409:  # duplicate replay
        # you have the choice: either raise an error, or return the existing replay id
        return r.json()['id']
    else:
        # raise an error for other status codes (50x, ...)
        r.raise_for_status()

In [5]:
for f in glob.glob('files/21_Aug/new/*'):
    upload(f, token)

In [6]:
data = requests.get(get_group_url, headers={
                    'Authorization': token})

In [7]:
data

<Response [200]>

In [8]:
linkat = []
for i in json.loads(data.text)['list']:
    linkat.append(i['link'])

matchat = []

for link in linkat:
    match_stats = json.loads(requests.get(link, headers={
                    'Authorization': token}).text)

    match = {}

    for i in ['blue', 'orange']:
        match[i] = {}
        match[i]['name'] = match_stats[i]['name']
        match[i]['match_id'] = match_stats['id']
        match[i]['match_date'] = match_stats['date']

        match[i]['players'] = []

        player_stats = match_stats[i]['players']
    
        for stat in player_stats:
            player = {}

            for f in ['id', 'name', 'car_name']:
                player[f] = stat[f]

            player['stats'] = {}
            player['stats']['movement'] = {}
            
            for f in ['core', 'demo']:
                player['stats'][f] = stat['stats'][f]

            for f in ['avg_speed', 'total_distance']:
                player['stats']['movement'][f] =  stat['stats']['movement'][f]


            match[i]['players'].append(player)

    matchat.append(match)

In [44]:
#match_stats

In [45]:
# x = matchat[1]
# y = matchat[3]
# z = copy.deepcopy(matchat[1])

# count = 0

# for f in ['blue', 'orange']:
#     for i in x[f]['players']:
#         for j in y[f]['players']:
#             if i['name'] == j['name']:
#                 for k in ['core', "demo", 'movement']:
#                     for key in i['stats'][k]:
#                         if key in j['stats'][k]:
#                             if key != 'avg_speed':
#                                 z[f]['players'][count]['stats'][k][key] = i['stats'][k][key] + j['stats'][k][key]
#                             else:
#                                 z[f]['players'][count]['stats'][k][key] = max(i['stats'][k][key], j['stats'][k][key])
#                         else:
#                             pass
#         count += 1
#     count = 0

# del matchat[3]
# matchat[1] = z
                

In [9]:
with open('21_aug.json', 'w') as f:
    json.dump({'matches' : matchat}, f)